In [ ]:
import torch

torch.cuda.is_available()

## Load monkey model

In [ ]:
# from laminr.neuron_models import monkey_v1

In [ ]:
# response_predicting_model = monkey_v1().to("cuda")

In [ ]:
# next(iter(response_predicting_model.parameters())).device

## Load simulated neuron model

In [ ]:
from laminr.neuron_models.simulated import neuron3_generator as neuron_model_generator

loc = [0.2, 0]
img_res = [100, 100]
neuron_model, rf_grid = neuron_model_generator(loc, img_res)
xx, yy = rf_grid

In [ ]:
from laminr.neuron_models.utils.simulated_model import ArbitraryMultiNeuronModel

In [ ]:
response_predicting_model = ArbitraryMultiNeuronModel([neuron_model]).to("cuda")

In [ ]:
response_predicting_model

## Generate MEIs

In [ ]:
import numpy as np
from laminr import get_mei_dict

In [ ]:
# neuron_idxs = [0]
# input_shape = [1, 93, 93]
# required_pixel_min = -1.7876
# required_pixel_max = 2.1919
# std = 0.15
# required_norm = std * np.sqrt(np.product(input_shape[-2:]))

neuron_idxs = [0]
input_shape = [1, 100, 100]
required_pixel_min = -10
required_pixel_max = 10
required_norm = 1

meis_dict = get_mei_dict(
    response_predicting_model,
    neuron_idxs,
    input_shape,
    required_pixel_min,
    required_pixel_max,
    required_norm=required_norm,
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(meis_dict[0]["mei"][0])

In [ ]:
plt.imshow(meis_dict[0]["mask"][0])

## Initialize the pipeline

In [ ]:
from laminr import InvarianceManifold

In [ ]:
invariance_manifold = InvarianceManifold(
    response_predicting_model,
    meis_dict,
    required_img_pixel_min=required_pixel_min,
    required_img_pixel_max=required_pixel_max,
    required_img_norm=required_norm,
)

In [ ]:
template_neuron_idx = 0

template = invariance_manifold.learn(
    template_neuron_idx
)

In [ ]:
from laminr import InvarianceManifold

In [ ]:
# NOTE that the pipeline config should include:
# - the shared args between template learning and matching based on the args listed above
# - args needed to learn the mei

pipeline_config = {
    ...
}

itlm = InvTempLearnMatch(model, **pipeline_config)

## Template learning

In [ ]:
template_neuron_idx = selected_neurons[0]

# the invariance manifold learning config should include:
# - the arguments that are not shared and are specific to learning the invariance manifold of the template neuron
invariance_manifold_learning_config = {
    ...
}
manifold1 = itlm.learn_invariance_manifold(template_neuron_idx, **invariance_manifold_learning_config)

## Template matching

In [ ]:
target_neuron_idx = selected_neurons[1]

# the invariance manifold learning config should include:
# - the arguments that are not shared and are specific to aligning the invariance manifold to the target neuron
manifold_alignment_config = {
    ...
}
alignment_score, transformed_manifold = pipeline.align_manifold(manifold1, target_neuron_idx, **manifold_alignment_config)